In [9]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt  
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve


In [10]:
X_train_gendered = pd.read_csv("X_train_gendered.csv")
X_dev_gendered = pd.read_csv("X_dev_gendered.csv")
X_test_gendered = pd.read_csv("X_test_gendered.csv")
y_train_gendered = pd.read_csv("y_train_gendered.csv")
y_dev_gendered = pd.read_csv("y_dev_gendered.csv")
y_test_gendered = pd.read_csv("y_test_gendered.csv")

X_train_not_gendered = pd.read_csv("X_train_not_gendered.csv")
X_dev_not_gendered = pd.read_csv("X_dev_not_gendered.csv")
X_test_not_gendered = pd.read_csv("X_test_not_gendered.csv")
y_train_not_gendered = pd.read_csv("y_train_not_gendered.csv")
y_dev_not_gendered = pd.read_csv("y_dev_not_gendered.csv")
y_test_not_gendered = pd.read_csv("y_test_not_gendered.csv")


In [63]:
def complete_matrix(df, vectorizer, tvec, gendered, include_gender = False):
    tf_idf_df = pd.DataFrame(tvec.toarray())
    
#     if not gendered:
#         original = df[["score", "num_comments"]]
#     else:
#         if not include_gender:
#             original = df[["score", "num_comments", "age"]]
#         else:
#             original = df[["score", "num_comments", "age", "gender"]]
#             original.replace({"gender": {"m": "M", "f": "F"}}, inplace=True)
#             original.replace({"gender": {"M": 1, "F": 0}}, inplace=True)
        
#     res = pd.concat([tf_idf_df, original], axis=1)
    
#     return res
    return tf_idf_df

In [12]:
y_train_gendered.head(2)


,Unnamed: 0,verdict
0,5661,not the asshole
1,12868,asshole


In [56]:
def model_and_scores(train_x, train_y, dev_x, dev_y, test_x, test_y, gendered, include_gender = False):
    model = LogisticRegression(multi_class = "multinomial", max_iter = 500)

    vectorizer = TfidfVectorizer(stop_words='english')
    
    tvec = vectorizer.fit_transform(train_x['body'].values.astype('U'))
    
    complete_train_x = complete_matrix(train_x, vectorizer, tvec, gendered, include_gender)
    model.fit(complete_train_x, train_y.verdict)
        
    complete_dev_x = complete_matrix(dev_x, vectorizer, vectorizer.transform(dev_x.body), gendered, include_gender)
    # print(complete_dev_x)
    dev_scores = accuracy_score(model.predict(complete_dev_x), dev_y.verdict)
    print(f"Dev Scores: {dev_scores}")
    
    complete_test_x = complete_matrix(test_x, vectorizer, vectorizer.transform(test_x.body), gendered, include_gender)
    test_scores = accuracy_score(model.predict(complete_test_x), test_y.verdict)
    print(f"Test Scores: {test_scores}")


In [ ]:
model_and_scores(X_train_not_gendered, y_train_not_gendered, X_dev_not_gendered, y_dev_not_gendered, X_test_not_gendered, y_test_not_gendered, False)

In [51]:
model_and_scores(X_train_gendered, y_train_gendered, X_dev_gendered, y_dev_gendered, X_test_gendered, y_test_gendered, True)

        0    1    2    3    4    5    6    7    8    9  ...  28281  28282  \
0     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0    0.0   
1     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0    0.0   
2     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0    0.0   
3     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0    0.0   
4     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0    0.0   
...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...    ...    ...   
1385  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0    0.0   
1386  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0    0.0   
1387  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0    0.0   
1388  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0    0.0   
1389  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0    0.0   

      28283  28284  28285  28286  28287  score  num_comments   age  
0     

In [61]:
model_and_scores(X_train_gendered, y_train_gendered, X_dev_gendered, y_dev_gendered, X_test_gendered, y_test_gendered, True, True)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:6662: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res[c] = res[c].replace(
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:6662: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res[c] = res[c].replace(


Dev Scores: 0.7906474820143885


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:6662: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res[c] = res[c].replace(


Test Scores: 0.7884892086330936


In [64]:
model_and_scores(X_train_gendered, y_train_gendered, X_dev_gendered, y_dev_gendered, X_test_gendered, y_test_gendered, False)


Dev Scores: 0.8057553956834532
Test Scores: 0.7913669064748201
